# Analysis of population and employment dynamics

In [ ]:
#nbd module
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from popemp.tools import Nbd
from popemp import data

nbd = Nbd('popemp')

In [ ]:
#nbd module
data_yr = data.pop().merge(data.emp(), 'left')

def data_ag(y0, y1):
    d = data_yr.query('year == @y0 or year == @y1').set_index(['st', 'cty', 'year'])[['pop', 'emp']].unstack('year')
    d = d[(d.notna() & (d > 0)).all(1)]
    d = d.stack(0)
    d = np.power(d[y1] / d[y0], 1/(y1-y0+1)).unstack().add_suffix('_agr_abs')
    d = (d - 1) * 100
    d = d.reset_index()

    d1 = d.query('cty == "000"').rename(columns={'pop_agr_abs': 'ref_pop_agr', 'emp_agr_abs': 'ref_emp_agr'})
    d = d.merge(d1.drop(columns='cty'), 'left')
    d.loc[d['cty'] == '000', ['ref_pop_agr', 'ref_emp_agr']] = d.loc[d['st'] == '00', ['ref_pop_agr', 'ref_emp_agr']].values
    d['pop_agr_rel'] = d['pop_agr_abs'] - d['ref_pop_agr']
    d['emp_agr_rel'] = d['emp_agr_abs'] - d['ref_emp_agr']
    return d

def color_from_gr(df, abs_rel):
    e = df['emp_agr_' + abs_rel]
    p = df['pop_agr_' + abs_rel]
    x = pd.Series(index=df.index, dtype='str')
    x[(p >= 0) & (e >= 0)] = 'red'
    x[(p >= 0) & (e <  0)] = 'green'
    x[(p <  0) & (e >= 0)] = 'orange'
    x[(p <  0) & (e <  0)] = 'blue'
    return x

In [ ]:
d = data_ag(2005, 2012).query('cty == "000"').copy()
fig, axes = plt.subplots(1, 2, figsize=(16, 8))
d['abs_cat'] = color_from_gr(d, 'abs')
d['rel_cat'] = color_from_gr(d, 'rel')

ax = axes[0]
d.plot.scatter('pop_agr_abs', 'emp_agr_abs', ax=ax, c='abs_cat')
xlim = abs(max(ax.get_xlim(), key=abs))
ax.set_xlim(-xlim, xlim)
ylim = abs(max(ax.get_ylim(), key=abs))
ax.set_ylim(-ylim, ylim)
ax.axvline(0, ls='-')
ax.axhline(0, ls='-')
ax.axvline(d['ref_pop_agr'].iloc[0], ls=':')
ax.axhline(d['ref_emp_agr'].iloc[0], ls=':')

ax = axes[1]
d.plot.scatter('pop_agr_rel', 'emp_agr_rel', ax=ax, c='rel_cat')
xlim = abs(max(ax.get_xlim(), key=abs))
ax.set_xlim(-xlim, xlim)
ylim = abs(max(ax.get_ylim(), key=abs))
ax.set_ylim(-ylim, ylim)
ax.axvline(0, ls='-')
ax.axhline(0, ls='-')

# Build this module

In [ ]:
nbd.nb2mod('analysis.ipynb')